In [33]:
import pymongo
import pandas as pd 
from pymongo import MongoClient
import pprint
from datetime import datetime
import time
import calendar

In [34]:
conn = 'mongodb://localhost:27017'
client = MongoClient(conn)

In [35]:
db = client.bike_data_db
bike_trip = db.bike_trip.find()

In [36]:
full_dict = []
for trip in bike_trip:
    full_dict.append(trip)
df = pd.DataFrame(full_dict)
df.head()

,_id,bike_id,duration,end_day,end_lat,end_lon,end_station,end_time,passholder_type,plan_duration,start_day,start_lat,start_lon,start_station,start_time,trip_id,trip_route_category,weekday
0,5c016831f596e114f49a3ba6,6279,8,2018-07-01,34.046520,-118.237411,3082,00:12:00,Monthly Pass,30,2018-07-01,34.035801,-118.233170,3058,00:04:00,94851140,One Way,Sunday
1,5c016831f596e114f49a3ba7,6518,8,2018-07-01,34.046520,-118.237411,3082,00:12:00,Flex Pass,365,2018-07-01,34.035801,-118.233170,3058,00:04:00,94851141,One Way,Sunday
2,5c016831f596e114f49a3ba8,4823,15,2018-07-01,34.165291,-118.150970,4174,00:24:00,Monthly Pass,365,2018-07-01,34.145248,-118.150070,4147,00:09:00,94851138,One Way,Sunday
3,5c016831f596e114f49a3ba9,6115,7,2018-07-01,34.147499,-118.148010,4162,00:29:00,Monthly Pass,30,2018-07-01,34.140999,-118.132088,4157,00:22:00,94851137,One Way,Sunday
4,5c016831f596e114f49a3baa,12055,35,2018-07-01,33.779819,-118.263023,3013,00:58:00,Walk-up,1,2018-07-01,33.779819,-118.263023,3013,00:23:00,94851136,Round Trip,Sunday


In [37]:
split = df["start_time"].str.split(":", n=1, expand=True)
df["hour"] = split[0]
df["minutes"] = split[1]

peak_hour = df[["start_station","hour","weekday","trip_id"]]
renamed = peak_hour.rename(columns={"start_station":"Station_ID"})
renamed.head()

,Station_ID,hour,weekday,trip_id
0,3058,00,Sunday,94851140
1,3058,00,Sunday,94851141
2,4147,00,Sunday,94851138
3,4157,00,Sunday,94851137
4,3013,00,Sunday,94851136


In [38]:
file = "static/data/metro-bike-share-stations-2018-10-19.csv"
stations = pd.read_csv(file)
stations.head()

,Station_ID,Station_Name,Go_live_date,Region,Status
0,3000,Virtual Station,2016-07-07,NaN,Active
1,3005,7th & Flower,7/7/2016,DTLA,Active
2,3006,Olive & 8th,7/7/2016,DTLA,Active
3,3007,5th & Grand,7/7/2016,DTLA,Active
4,3008,Figueroa & 9th,7/7/2016,DTLA,Active


In [39]:
merged_df = pd.merge(renamed, stations, on="Station_ID", how="outer")
merged_df.head()

,Station_ID,hour,weekday,trip_id,Station_Name,Go_live_date,Region,Status
0,3058,00,Sunday,94851140.0,Industrial St & Mateo St,7/7/2016,DTLA,Active
1,3058,00,Sunday,94851141.0,Industrial St & Mateo St,7/7/2016,DTLA,Active
2,3058,17,Sunday,94973454.0,Industrial St & Mateo St,7/7/2016,DTLA,Active
3,3058,18,Sunday,94973447.0,Industrial St & Mateo St,7/7/2016,DTLA,Active
4,3058,19,Sunday,94980682.0,Industrial St & Mateo St,7/7/2016,DTLA,Active


In [40]:
merged_df = merged_df.dropna(how="any")
merged_df.head()

,Station_ID,hour,weekday,trip_id,Station_Name,Go_live_date,Region,Status
0,3058,00,Sunday,94851140.0,Industrial St & Mateo St,7/7/2016,DTLA,Active
1,3058,00,Sunday,94851141.0,Industrial St & Mateo St,7/7/2016,DTLA,Active
2,3058,17,Sunday,94973454.0,Industrial St & Mateo St,7/7/2016,DTLA,Active
3,3058,18,Sunday,94973447.0,Industrial St & Mateo St,7/7/2016,DTLA,Active
4,3058,19,Sunday,94980682.0,Industrial St & Mateo St,7/7/2016,DTLA,Active


In [49]:
live_stations = merged_df[["Station_Name","hour","weekday"]]
filter_one = live_stations.loc[live_stations["Station_Name"] == "Industrial St & Mateo St",:]

In [54]:
filter_two = filter_one.loc[filter_one["weekday"] == "Monday", :]
grouped = filter_two.groupby(["hour"]).count()
reset_index = grouped.reset_index()

In [55]:
reset_index.to_json(orient='records')

'[{"hour":"00","Station_Name":5,"weekday":5},{"hour":"01","Station_Name":3,"weekday":3},{"hour":"07","Station_Name":5,"weekday":5},{"hour":"08","Station_Name":6,"weekday":6},{"hour":"09","Station_Name":7,"weekday":7},{"hour":"10","Station_Name":1,"weekday":1},{"hour":"11","Station_Name":2,"weekday":2},{"hour":"12","Station_Name":9,"weekday":9},{"hour":"13","Station_Name":4,"weekday":4},{"hour":"14","Station_Name":8,"weekday":8},{"hour":"15","Station_Name":2,"weekday":2},{"hour":"16","Station_Name":6,"weekday":6},{"hour":"17","Station_Name":15,"weekday":15},{"hour":"18","Station_Name":14,"weekday":14},{"hour":"19","Station_Name":1,"weekday":1},{"hour":"20","Station_Name":6,"weekday":6},{"hour":"21","Station_Name":8,"weekday":8},{"hour":"22","Station_Name":3,"weekday":3}]'